In [ ]:
#verzija BMB: 12.4.2022---------------IMAGES-zfotovse___.csv--------------------basic kNN with PM-----------------------------
#1) load data IMAGES	2) define PM	4) lmnn	4) knn	5) CV


#import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.model_selection import cross_val_score   #for crossvalidation
from sklearn.model_selection import StratifiedKFold

from pylmnn import LargeMarginNearestNeighbor as LMNN
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score    #for confusion matrix
from sklearn import datasets, neighbors, metrics   # Import datasets, classifiers and performance metrics

from sklearn.preprocessing import Normalizer	#for normalization

from sklearn.neighbors import DistanceMetric
from sklearn.preprocessing import StandardScaler    ##for standardization-Gaussian with zero mean and unit variance.

#BMB
import numpy as np
from sklearn.model_selection import train_test_split
import numpy as np
#import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import matplotlib.ticker as ticker
#%matplotlib inline


df = pd.read_csv('fotovseEdgeresample150.csv', header=None)
df.iloc[:,80].unique() # 
df.count(axis=0, level=None, numeric_only=False)

train_df = pd.read_csv('fotovseEdgeresample150.csv', header=None)
X = np.array(train_df.iloc[:, :-1])	#all rows & all column except last one
#print(train_data)
y = np.array(train_df.iloc[:, -1])   #all rows & only last column

#BMB define train and test splits
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=52)
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.6, random_state=52)

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, train_size=0.9) # create a validation set from the already splited training set

X.shape
print(f"Mn X {X.shape}")

# BMB
X_train.shape
X_test.shape 
y_train.shape 
y_test.shape


#BMB Not sure if normalisation oges in steps

transformer = Normalizer().fit(X_train)
X_train = transformer.transform(X_train)

transformer = Normalizer().fit(X_test)
X_test = transformer.transform(X_test)

transformer = Normalizer().fit(X_valid)
X_valid = transformer.transform(X_valid)


#Before making any actual predictions, it is always a good practice to scale the features so that all of them can be uniformly evaluated = standardization
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)

scaler = StandardScaler()
scaler.fit(X_test)
X_test = scaler.transform(X_test)

scaler = StandardScaler()
scaler.fit(X_valid)
X_valid = scaler.transform(X_valid)


# LMNN: algorithm for metric learning---------------------after cross validation
# Set up the hyperparameters
k_train, k_test, max_iter = 2, 2, 2000	#n_components=X.shape[1], za bpp:640


#@staticmethod
def poin_distanceBMB(row1, row2):
    #print('len(row1) = ',len(row1))
    #print('len(row1) = ',len(row1))
    distance = 0.0
    d = 0.0
    norm1 = 0.0
    norm2 = 0.0
    
    d = (row1 - row2)*(row1 - row2)
    d = d.sum()
    
    norm1 = row1 * row1
    norm1 = norm1.sum()
    
    norm2 = row2 * row2
    norm2 = norm2.sum()
    
    m = 1 + 2*d/((1-norm1)*(1-norm2))
    distance = np.log(1+m + np.sqrt(m*m - 1)) # BMB dodano 1+
    
    return(distance)
    
    
# BMB: how do we define the diference between the nominal values? Is that also included, for the last value in each raw?


# Instantiate the metric learner
lmnn = LMNN(n_neighbors=k_train, max_iter=max_iter)

# Train the metric learner
lmnn.fit(X_valid, y_valid)

# Create a classifier: a knn with PM
acc1 = []
acc1_train = []
accuracies = []
for i in range(1,11):
	knn = neighbors.KNeighborsClassifier(n_neighbors=i, metric='pyfunc', metric_params={"func":poin_distanceBMB}).fit(X_train,y_train) 	#to add new metric: metric='pyfunc'
	#for testing
	#knn = neighbors.KNeighborsClassifier(n_neighbors=5, algorithm='brute', metric='euclidean')    #algorithm='ball_tree' - for a custom metric

	# Learn the digits/iris on the train subset - fit the model
	# BMB: Mi ni jasno kako funkciji lmnn damo Poankare distance?
	knn.fit(lmnn.transform(X_train),y_train)

	predicted = knn.predict(lmnn.transform(X_test))
	predicted_train= knn.predict(lmnn.transform(X_train))

	acc1.append(metrics.accuracy_score(y_test, predicted))
	acc1_train.append(metrics.accuracy_score(y_train, predicted_train))

	accc = accuracy_score(y_test, predicted)
	accuracies.append(accc)
	print("k = "+str(i)+"; Accuracy: "+str(accc))
	

""" 
    # Predict the value of the digit on the test subset  and calculate the accuracy
    predicted = knn.predict(lmnn.transform(X_test))
    acc1.append(metrics.accuracy_score(y_test, predicted))
    print("Accuracy test")
    print(acc1.append(metrics.accuracy_score(y_test, predicted)))
    #acc1.append(knn.score(y_test, predicted))
    
    # Predict the value of the digit on the test subset  and calculate the accuracy of the trining set
    predicted_train = knn.predict(lmnn.transform(X_train))   
    acc1_train.append(metrics.accuracy_score(y_train, predicted_train))
    print("Accuracy train")
    print(acc1_train.append(metrics.accuracy_score(y_train, predicted_train)))

"""

















